# package

In [67]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *
import os
import polars as pl
import numpy as np
import pandas as pd
from functools import lru_cache
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

In [132]:
task = 'task3'

# Read data 

In [30]:
# ls ../data/sub_files/ | grep test4task3

In [68]:
task2 = pl.scan_parquet('../data/sub_files/task2_test4task3_task2_rank_lgbm_v2.parquet')
task1 = pl.scan_parquet('../data/sub_files/task1_test4task3_task1_rank_lgbm_v10.parquet')


test4task3_pl = pl.scan_parquet(os.path.join('../data/raw_data_session_id/', 'sessions_test_task3.parquet')).with_columns(pl.col('prev_items').apply(str2list))


In [69]:
test4task3_pl.collect().shape

(56422, 3)

In [70]:
task2.head().collect()

prev_items,locale,session_id,next_item_prediction
list[str],str,i64,list[str]
"[""B0BF9JMVDG"", ""B01ET9V90M""]","""ES""",4365996,"[""B00XA0GJSE""]"
"[""B09QQG85HM"", ""B09J4T4JF5""]","""ES""",4365997,"[""B09K7TDY1H""]"
"[""B09NSKDG4K"", ""B09YY6J1ZM""]","""ES""",4365998,"[""B09XM6Z7VY""]"
"[""B09B7NYDJ7"", ""B09B7NYDJ7""]","""ES""",4365999,"[""B014EWSGX2""]"
"[""B0B6J17LK4"", ""B0B6R7X6GY"", ""B07HXY5SGH""]","""ES""",4366000,"[""B09DXT3NGZ""]"


In [71]:
task1.head().collect()

prev_items,locale,session_id,next_item_prediction
list[str],str,i64,list[str]
"[""B01D37JZDO"", ""B09798DT5N""]","""DE""",4372418,"[""B07QV6GZ6P""]"
"[""B09TPHS4J1"", ""B09TPHGHR8""]","""DE""",4372419,"[""B09TPHBD98""]"
"[""B09CPRS6QK"", ""B09XMGPTZ2""]","""DE""",4372420,"[""B093ZXQQ9Y""]"
"[""3785586620"", ""3809439908""]","""DE""",4372421,"[""3741525065""]"
"[""B09Q3DCGW3"", ""B09Q3C5Z33""]","""DE""",4372422,"[""B09Q3C3CKB""]"


In [88]:
res = pl.concat([task1, task2], how='vertical')
product_pl = (pl.scan_parquet(os.path.join('../data/raw_data_session_id/', 
                                          'products_train.parquet'))
                  # .with_columns(
                  #     pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                  #       .then(2)
                  #       .otherwise(3).alias('locale')
                  # )
             )

In [89]:
# product_pl.select(['id', 'locale', 'title']).head().collect()

In [90]:
# res.head().collect()

In [91]:
# test4task3_pl.schema

In [93]:
predictions_step1 = (
    test4task3_pl.join(
        res.select(['session_id', 'locale', 'next_item_prediction'])
        , how='left'
        , on=['session_id', 'locale']
    )
    .explode('next_item_prediction')
    .with_columns(
        pl.when(pl.col('next_item_prediction').is_null()).then(pl.col('prev_items').arr.get(-1)).otherwise(pl.col('next_item_prediction'))
        .alias('next_item_prediction')
    )
)#.head().collect()

In [94]:
# predictions.head().collect()

In [114]:
predictions = (
    predictions_step1
        .join(product_pl.select(['id', 'locale', 'title']), how='left', right_on=['id', 'locale']
                 , left_on=['next_item_prediction', 'locale'])
        .with_columns(pl.col('title').alias('next_item_prediction').cast(pl.Utf8))
)#.head().collect()

In [115]:
assert predictions.filter(pl.col('title').is_null()).collect().shape[0] == 0

AssertionError: 

In [116]:
assert test4task3_pl.collect().shape[0] == predictions.collect().shape[0]

In [117]:
# product_pl.filter(pl.col('id')=='B085NXQL7F').collect()

In [118]:
# test4task3_pl.filter(pl.col('session_id')==4420531).collect()

In [120]:
predictions.head().collect()

prev_items,locale,session_id,next_item_prediction,title
list[str],str,i64,str,str
"[""B0BF9JMVDG"", ""B01ET9V90M""]","""ES""",4365996,"""Coca-Cola Zero…","""Coca-Cola Zero…"
"[""B09QQG85HM"", ""B09J4T4JF5""]","""ES""",4365997,"""Xiaomi Redmi B…","""Xiaomi Redmi B…"
"[""B09NSKDG4K"", ""B09YY6J1ZM""]","""ES""",4365998,"""Yisica Correa …","""Yisica Correa …"
"[""B09B7NYDJ7"", ""B09B7NYDJ7""]","""ES""",4365999,"""Rowenta Compac…","""Rowenta Compac…"
"[""B0B6J17LK4"", ""B0B6R7X6GY"", ""B07HXY5SGH""]","""ES""",4366000,"""STN Fiambrera …","""STN Fiambrera …"


In [119]:
predictions.filter(pl.col('title').is_null()).collect()

prev_items,locale,session_id,next_item_prediction,title
list[str],str,i64,str,str
"[""B01BFA2BFK"", ""B07DS9F1Z1""]","""ES""",4366081,null,null
"[""B09NLQC628"", ""B09Q5QPDNQ""]","""ES""",4366435,null,null
"[""B07Q6VCP5B"", ""B08VM7LY4W"", … ""B07Q6VCP5B""]","""ES""",4366646,null,null
"[""B09Q5QPDNQ"", ""B09NLQC628"", ""B09Q5QPDNQ""]","""ES""",4366723,null,null
"[""B07Q71PGTZ"", ""B07Q4SDMZ4"", ""B07Q9PWVTJ""]","""ES""",4368575,null,null
"[""B07Q2J5XMK"", ""B07Q2J5XMK""]","""ES""",4369978,null,null
"[""B07XGBFYVG"", ""B07XGBFYVG""]","""ES""",4370445,null,null
"[""B009P890YA"", ""B0043A0094"", … ""B009P890YA""]","""ES""",4371883,null,null
"[""B09MBL23YZ"", ""B083KTWP8J""]","""ES""",4371902,null,null


In [121]:

# predictions.select('locale').collect().to_series().value_counts()

In [122]:
predictions_df = predictions.collect().to_pandas()

In [123]:
predictions_df['next_item_prediction'] = predictions_df['next_item_prediction'].astype('str')

# Validate predictions ✅

In [127]:
def check_predictions(predictions, test_sessions):
    """
    These tests need to pass as they will also be applied on the evaluator
    """
    test_locale_names = test_sessions['locale'].unique()
    for locale in test_locale_names:
        sess_test = test_sessions.query(f'locale == "{locale}"')
        preds_locale =  predictions[predictions['locale'] == sess_test['locale'].iloc[0]]
        assert sorted(preds_locale.index.values) == sorted(sess_test.index.values), f"Session ids of {locale} doesn't match"
        assert predictions['next_item_prediction'].apply(lambda x: isinstance(x, str)).all(), "Predictions should all be strings"

In [128]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56422 entries, 0 to 56421
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   prev_items            56422 non-null  object
 1   locale                56422 non-null  object
 2   session_id            56422 non-null  int64 
 3   next_item_prediction  56422 non-null  object
 4   title                 56379 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


In [133]:
check_predictions(predictions_df, test_sessions=test4task3_pl.collect().to_pandas())

In [136]:
sub_file = f'submission_{task}.parquet'

In [142]:
! ls -al {sub_file}

-rw-r--r-- 1 jupyter jupyter 12565524 Jun  8 19:50 submission_task3.parquet


In [138]:
# Its important that the parquet file you submit is saved with pyarrow backend
predictions_df.to_parquet(sub_file, engine='pyarrow')

In [141]:
f'submission_{task}.parquet'

'submission_task3.parquet'

## Submit to AIcrowd 🚀

In [145]:
# You can submit with aicrowd-cli, or upload manually on the challenge page.
# !aicrowd submission create -c task-3-next-product-title-generation -f {sub_file}